In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Subgraph

1. 서브 그래프를 직접 호출하는 노드 추가
2. 하위 그래프를 호출하는 함수가 있는 노드를 추가

In [2]:
# 1. 서브 그래프를 직접 호출하는 노드 추가
from typing import TypedDict
from langgraph.graph import START, StateGraph

class State(TypedDict):
    foo: str  

class SubgraphState(TypedDict):
    foo: str  
    bar: str


In [4]:
# 서브 그래프 정의
def subgraph_node(state: SubgraphState):
    return {"foo": state["foo"] + "bar"}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("subgraph_node", subgraph_node)
subgraph_builder.add_edge(START, "subgraph_node")
subgraph = subgraph_builder.compile()


# 부모 그래프 정의
builder = StateGraph(State)
builder.add_node("subgraph", subgraph)
builder.add_edge(START, "subgraph")
graph = builder.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
initial_state = {"foo": "hello"}
result = graph.invoke(initial_state)
print(f"Result: {result}")  

---
### 두번째 방법
2. 하위 그래프를 호출하는 함수가 있는 노드를 추가

In [8]:
from typing import TypedDict
from langgraph.graph import START, StateGraph


class State(TypedDict):
    foo: str


class SubgraphState(TypedDict):
    # 부모의 그래프 상태와 키를 공유하지 않음
    bar: str
    baz: str

In [9]:
# 서브 그래프 정의
def subgraph_node(state: SubgraphState):
    return {"bar": state["bar"] + "baz"}


subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node("subgraph_node", subgraph_node)
subgraph_builder.add_edge(START, "subgraph_node")
subgraph = subgraph_builder.compile()

In [11]:
# 서브 그래프를 호출하는 부모 그래프 정의
def node(state: State):
    response = subgraph.invoke({"bar": state["foo"]})
    # 부모 상태로 응답값을 변환
    return {"foo": response["bar"]}


builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")
graph = builder.compile()


In [ ]:
initial_state = {"foo": "hello"}
result = graph.invoke(initial_state)
print(
    f"Result: {result}"
)  # Should transform foo->bar, append "baz", then transform bar->foo

### Multi-agent

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from langchain.tools import tool
from typing import TypedDict, List, Annotated
from langgraph.graph.message import add_messages  # ✅ 메시지 병합 함수

# 0. 명시적 상태 스키마 정의
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# 1. 모델 정의
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. 도구 정의
@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def web_search(query: str) -> str:
    """Return company headcounts based on a simulated web search."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

# 3. 에이전트 생성
math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

# 4. Supervisor 노드 함수 정의
def supervisor_node(state: AgentState) -> AgentState:
    last_message = state["messages"][-1].content.lower()

    if "headcount" in last_message:
        # 첫 요청 → 리서치
        if "combined" in last_message or "total" in last_message:
            if "research_done" not in state:
                return {"__next__": "research_expert", "research_done": True}
            else:
                return {"__next__": "math_expert"}
    return {"__next__": END}

# 5. 그래프 구성
graph = StateGraph(AgentState)
graph.add_node("supervisor", supervisor_node)
graph.add_node("math_expert", math_agent)
graph.add_node("research_expert", research_agent)

graph.set_entry_point("supervisor")
graph.add_edge("supervisor", "math_expert")
graph.add_edge("supervisor", "research_expert")
graph.add_edge("math_expert", END)
graph.add_edge("research_expert", END)

# 6. 컴파일 및 실행
app = graph.compile()

# 7. 실행
output = app.invoke({
    "messages": [HumanMessage(content="2024년 FAANG 기업의 총 직원 수는 얼마입니까? 전체 하나의 수로 응답하세요.")]
})

# 결과 출력
from langchain_core.messages import get_buffer_string
print(get_buffer_string(output["messages"]))

last_msg = output["messages"][-1]
print("🧾 Final answer:\n", last_msg.content)

In [ ]:
pip install -U langgraph langgraph-supervisor langchain